## Cleaning Data with PySpark

Working with data is tricky - working with millions or even billions of rows is worse. Did you receive some data processing code written on a laptop with fairly pristine data? Chances are you’ve probably been put in charge of moving a basic data process from prototype to production. You may have worked with real world datasets, with missing fields, bizarre formatting, and orders of magnitude more data. Even if this is all new to you, this notebook helps you learn what’s needed to prepare data processes using Python with Apache Spark. 

### Defining a schema

- Creating a defined schema helps with data quality and import performance. As mentioned during the lesson, we'll create a simple schema to read in the following columns:

Name
Age
City
The `Name` and `City` columns are `StringType()` and the `Age` column is an `IntegerType()`

In [40]:
# Import the pyspark.sql.types library ->Import * from the pyspark.sql.types library.
from pyspark.sql.types import *

# Define a new schema using the StructType method
people_schema = StructType([
  # Define a StructField for each field -> Define a StructField for name, age, and city. Each field should correspond to the correct datatype and not be nullable.
  StructField('name', StringType(), False),
  StructField('age', IntegerType(), False),
  StructField('city', StringType(), False),
])

In [41]:
# to build a new SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext

spark= SparkSession.builder.getOrCreate()
    
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [42]:
# check the saved table in spark
spark.catalog.listTables()

[]

### Using lazy processing

- Lazy processing operations will usually return in about the same amount of time regardless of the actual quantity of data. Remember that this is due to Spark not performing any transformations until an action is requested.

- For this exercise, we'll be defining a Data Frame (aa_dfw_df) and add a couple transformations. Note the amount of time required for the transformations to complete when defined vs when the data is actually queried. These differences may be short, but they will be noticeable. When working with a full Spark cluster with larger quantities of data the difference will be more apparen

In [43]:
# Load the CSV file
from pyspark.sql.functions import lower
aa_dfw_df = spark.read.csv('/Users/ssamilozkan/Desktop/pyspark/cleaning_data_with_pyspark/dataset/AA_DFW_2014_Departures_Short.csv', header=True)

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df.show(3)


+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2014|         0005|                          519|    hnl|
|       01/01/2014|         0007|                          505|    ogg|
|       01/01/2014|         0035|                          174|    slc|
+-----------------+-------------+-----------------------------+-------+
only showing top 3 rows



### Saving a DataFrame in Parquet format

- When working with Spark, you'll often start with CSV, JSON, or other data sources. This provides a lot of flexibility for the types of data to load, but it is not an optimal format for Spark. The Parquet format is a columnar data store, allowing Spark to use predicate pushdown. This means Spark will only process the data necessary to complete the operations you define versus reading the entire dataset. This gives Spark more flexibility in accessing the data and often drastically improves performance on large datasets.

- In this exercise, we're going to practice creating a new Parquet file and then process some data from it.

- The spark object and the df1 and df2 DataFrames have been setup for you.

In [44]:
df1 = spark.read.csv('/Users/ssamilozkan/Desktop/pyspark/cleaning_data_with_pyspark/dataset/AA_DFW_2014_Departures_Short.csv', header=True)
df2 = spark.read.csv('/Users/ssamilozkan/Desktop/pyspark/cleaning_data_with_pyspark/dataset/AA_DFW_2015_Departures_Short.csv', header=True)
print(type(df1))
print(type(df2))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [45]:
# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)

# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())

df1 Count: 157198
df2 Count: 146558


303756


### SQL and Parquet

- Parquet files are perfect as a backing data store for SQL queries in Spark. While it is possible to run the same queries directly via Spark's Python functions, sometimes it's easier to run SQL queries alongside the Python options.

- For this example, we're going to read in the Parquet file we created in the last exercise and register it as a SQL table. Once registered, we'll run a quick query against the table (aka, the Parquet file).

- The spark object and the AA_DFW_ALL.parquet file are available for you automatically

In [46]:
# check the saved table in spark
spark.catalog.listTables()

[]

In [47]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet('AA_DFW_ALL.parquet')
flights_df.show(3)


+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2015|         0005|                HNL|                          526|
|       01/01/2015|         0007|                OGG|                          517|
|       01/01/2015|         0023|                SFO|                          233|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 3 rows



In [48]:
flights_df= flights_df.withColumnRenamed("Actual elapsed time (Minutes)", "flight_duration")
flights_df.show(3)

+-----------------+-------------+-------------------+---------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|flight_duration|
+-----------------+-------------+-------------------+---------------+
|       01/01/2015|         0005|                HNL|            526|
|       01/01/2015|         0007|                OGG|            517|
|       01/01/2015|         0023|                SFO|            233|
+-----------------+-------------+-------------------+---------------+
only showing top 3 rows



In [49]:
# Register the temp table
flights_df.createOrReplaceTempView('flights')

# check the saved table in spark
spark.catalog.listTables()

[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [50]:
type(flights_df)
flights_df.show(3)

+-----------------+-------------+-------------------+---------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|flight_duration|
+-----------------+-------------+-------------------+---------------+
|       01/01/2015|         0005|                HNL|            526|
|       01/01/2015|         0007|                OGG|            517|
|       01/01/2015|         0023|                SFO|            233|
+-----------------+-------------+-------------------+---------------+
only showing top 3 rows



In [51]:
flights_df.printSchema()

root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Flight Number: string (nullable = true)
 |-- Destination Airport: string (nullable = true)
 |-- flight_duration: string (nullable = true)



In [54]:
# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

The average flight time is: 143
